# Rewrite to Other Kinds of Frameworks

<br />

I will rewrite codes of TensorFlow that I created on the sprint 14 that handle 4 kinds of datasets to the other kind of frameworks, Keras.

## [Task 6] Rewrite to PyTorch

### Fit Iris Dataset by Keras (Multinominal Classification)

In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
import torch
from torch.autograd import Variable
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

In [2]:
# Read dataset

dataset_path = "Iris.csv"
df = pd.read_csv(dataset_path)

In [3]:
df

,Id,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Species
0,1,5.1,3.5,1.4,0.2,Iris-setosa
1,2,4.9,3.0,1.4,0.2,Iris-setosa
2,3,4.7,3.2,1.3,0.2,Iris-setosa
3,4,4.6,3.1,1.5,0.2,Iris-setosa
4,5,5.0,3.6,1.4,0.2,Iris-setosa
5,6,5.4,3.9,1.7,0.4,Iris-setosa
6,7,4.6,3.4,1.4,0.3,Iris-setosa
7,8,5.0,3.4,1.5,0.2,Iris-setosa
8,9,4.4,2.9,1.4,0.2,Iris-setosa
9,10,4.9,3.1,1.5,0.1,Iris-setosa


In [4]:
y = df["Species"]
X = df.loc[:, ["SepalLengthCm", "SepalWidthCm", "PetalLengthCm", "PetalWidthCm"]]

y = np.array(y)
X = np.array(X)

In [5]:
# Change labels to values

y[y=='Iris-setosa'] = 0
y[y=='Iris-versicolor'] = 1
y[y=='Iris-virginica'] = 2

y = y.astype(np.int)[:, np.newaxis]

In [6]:
# One hot encoding

# Initialize
ohe = OneHotEncoder(handle_unknown='ignore', sparse=False)

# Fit
y_one_hot = ohe.fit_transform(y)

In [7]:
# Split the dataframe into train and test datasets

X_train, X_test, y_train, y_test = train_test_split(X, y_one_hot, test_size=0.2, random_state=0)

In [8]:
# Split the train dataset into train and validation datasets again

X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=0)

In [24]:
# Change to tensor

X_train = torch.Tensor(X_train)
X_test = torch.Tensor(X_test)
X_val = torch.Tensor(X_val)
y_train = torch.LongTensor(y_train)
y_test = torch.LongTensor(y_test)
y_val = torch.LongTensor(y_val)

In [18]:
# Set hyperparameters

LEARNING_RATE = 0.01
BATCH_SIZE = 10
NUM_EPOCHS = 10

In [19]:
N_HIDDEN1 = 50
N_HIDDEN2 = 100
n_input = X_train.shape[1]
#n_samples = X_train.shape[0]
N_CLASSES = 3

In [20]:
class NeuralNet(nn.Module):
    def __init__(self):
        super(NeuralNet, self).__init__()
        self.fc1 = nn.Linear(n_input, N_HIDDEN1)
        self.fc2 = nn.Linear(N_HIDDEN1, N_HIDDEN2)
        self.fc3 = nn.Linear(N_HIDDEN2, N_CLASSES)
    
    
    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.log_softmax(self.fc3(x))
        return x

In [21]:
model = NeuralNet()
optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)
criterion = nn.CrossEntropyLoss()

In [27]:
for i in range(NUM_EPOCHS):
    inputs, target = Variable(X_train), Variable(y_train)
    optimizer.zero_grad()
    outputs = model(inputs)
    loss = criterion(outputs, target)
    loss.backward()
    optimizer.step()

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:12: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  if sys.path[0] == '':


RuntimeError: multi-target not supported at /Users/soumith/mc3build/conda-bld/pytorch_1549597882250/work/aten/src/THNN/generic/ClassNLLCriterion.c:21